In [3]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time


device = torch.device('cuda')

In [6]:
train_png = sorted(glob('./data/open/train/*.png'))
test_png = sorted(glob('./data/open/test/*.png'))

In [14]:
train_y = pd.read_csv('./data/open/train_df.csv')
train_labels = train_y['label']

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

hazelnut-good                 391
screw-good                    320
carpet-good                   280
pill-good                     267
grid-good                     264
                             ... 
transistor-bent_lead            5
transistor-damaged_case         5
cable-cut_outer_insulation      5
wood-liquid                     5
wood-color                      4
Name: label, Length: 88, dtype: int64